In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
%matplotlib inline

# Setup and cleaning

In [26]:
#reading all files
FR = pd.read_csv("FRvideos.csv")
CA = pd.read_csv("CAvideos.csv")
DE = pd.read_csv("DEvideos.csv")
GB = pd.read_csv("GBvideos.csv")
IN = pd.read_csv("INvideos.csv")
# Using only same alphabet 
#JP = pd.read_csv("JPvideos.csv", encoding='iso-8859-1') 
#KR = pd.read_csv("KRvideos.csv", encoding='iso-8859-1') 
#MX = pd.read_csv("MXvideos.csv", encoding='iso-8859-1') 
#RU = pd.read_csv("RUvideos.csv", encoding='iso-8859-1') 
US = pd.read_csv("USvideos.csv")

In [27]:
# Read json with categories
jsFR = pd.read_json("FR_category_id.json")
jsCA = pd.read_json("CA_category_id.json")
jsDE = pd.read_json("DE_category_id.json")
jsGB = pd.read_json("GB_category_id.json")
jsIN = pd.read_json("IN_category_id.json")
jsUS = pd.read_json("US_category_id.json")

list_js = [jsFR, jsCA, jsDE, jsGB, jsIN, jsUS]

#convert category id'S to their names
for l in range(len(list_js)):
    for i in range(len(list_js[l])):
        #list_country[l]["category_id"].astype(str)
        FR.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)
        CA.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)
        DE.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)
        GB.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)
        IN.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)
        US.category_id.replace(int(list_js[l]["items"][i]["id"]), list_js[l]["items"][i]["snippet"]["title"], inplace=True)

In [28]:
#adding nation as a column
FR["country"] = 'FR'
CA["country"] = 'CA'
GB["country"] = 'GB'
IN["country"] = 'IN'
US["country"] = 'US'

In [29]:
#concat all together, cleaning, formatting and reset index
youtube = pd.concat([FR,CA,GB,IN,US])
youtube.reset_index(drop=True, inplace=True)
youtube.drop(["video_id","thumbnail_link"], axis=1, inplace=True)
youtube.publish_time = youtube.publish_time.astype('datetime64[ns]')
youtube['trending_date']='20'+youtube['trending_date']
youtube['trending_date'].replace('\.','',regex=True, inplace=True)
youtube['trending_date'] = pd.to_datetime(youtube['trending_date'], format='%Y%d%m')
youtube.head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,2017-11-14,Malika LePen : Femme de Gauche - Trailer,Le Raptor Dissident,Entertainment,2017-11-13 17:32:55,"Raptor""|""Dissident""|""Expliquez""|""moi""|""cette""|...",212702,29282,1108,3817,False,False,False,Dimanche.\n18h30.\nSoyez présents pour la vidé...,FR
1,2017-11-14,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",Le Labo,Entertainment,2017-11-12 15:00:02,[none],432721,14053,576,1161,False,False,False,Le jeu de société: https://goo.gl/hhG1Ta\n\nGa...,FR
2,2017-11-14,DESSINS ANIMÉS FRANÇAIS VS RUSSES 2 - Daniil...,Daniil le Russe,Comedy,2017-11-13 17:00:38,"cartoon""|""pokémon""|""école""|""ours""|""мультфильм",482153,76203,477,9580,False,False,False,Une nouvelle dose de dessins animés français e...,FR
3,2017-11-14,PAPY GRENIER - METAL GEAR SOLID,Joueur Du Grenier,Gaming,2017-11-12 17:00:02,"Papy grenier""|""Metal Gear Solid""|""PS1""|""Tirage...",925222,85016,550,4303,False,False,False,"Nouvel ,épisode de Papy Grenier ! Ce mois-ci o...",FR
4,2017-11-14,QUI SAUTERA LE PLUS HAUT ? (VÉLO SKATE ROLLER ...,Aurelien Fontenoy,Sports,2017-11-13 16:30:03,"vélo""|""vtt""|""bmx""|""freestyle""|""bike""|""mtb""|""di...",141695,8091,72,481,False,False,False,Sauts à plus de 4 mètres de haut dans un tramp...,FR


# Individual tag count per category for the circular packing

In [30]:
# select only meaningful columns and split then explode the tags into several rows
df = youtube[['tags','category_id','country']].copy()
df['tags'] = df['tags'].str.split('|')
df = df.explode('tags')
df['tags'] = df['tags'].map(lambda x: x.lstrip('"').rstrip('"'))
df = df[df.tags!='[none]']
df = df[df.tags!='']
df.reset_index(inplace=True, drop=True)
df.rename(columns={"category_id": "category", 'country':'count'}, inplace=True)
df

,tags,category,count
0,Raptor,Entertainment,FR
1,Dissident,Entertainment,FR
2,Expliquez,Entertainment,FR
3,moi,Entertainment,FR
4,cette,Entertainment,FR
...,...,...,...
3595053,ending explained,Film & Animation,US
3595054,call of duty,Gaming,US
3595055,cod,Gaming,US
3595056,activision,Gaming,US


In [31]:
# groupby to get number of times a tag is used per category
grouped = df.groupby(["category", "tags"], as_index=False).count()

In [32]:
grouped

,category,tags,count
0,Autos & Vehicles,Toyota,7
1,Autos & Vehicles,#25,1
2,Autos & Vehicles,#737,1
3,Autos & Vehicles,#AUTO,1
4,Autos & Vehicles,#BMW,1
...,...,...,...
453455,Travel & Events,肉まん,9
453456,Travel & Events,茶懐石,13
453457,Travel & Events,西洋料理 島,2
453458,Travel & Events,신포참닭강정,2


In [33]:
# save CSV
grouped.to_csv("tags.csv", index=False)

# Top tags per category for the circular bar plot

In [34]:
# rotate previous dataframe
pivot = grouped.pivot(index='tags', columns='category', values = 'count')

In [35]:
pivot

category,Autos & Vehicles,Comedy,Education,Entertainment,Film & Animation,Gaming,Howto & Style,Movies,Music,News & Politics,Nonprofits & Activism,People & Blogs,Pets & Animals,Science & Technology,Shows,Sports,Travel & Events
tags,,,,,,,,,,,,,,,,,
#Freeticket,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#Jaisimha,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#JanaSenaParty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#MahaaNews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
👩🏻,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
💅,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
😂,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# setup the columns for the csv: catx are the most used tags for a given category
cols = pivot.columns.tolist()
for i in range(len(cols)):
    cols.append('cat'+str(i))
cols

['Autos & Vehicles',
 'Comedy',
 'Education',
 'Entertainment',
 'Film & Animation',
 'Gaming',
 'Howto & Style',
 'Movies',
 'Music',
 'News & Politics',
 'Nonprofits & Activism',
 'People & Blogs',
 'Pets & Animals',
 'Science & Technology',
 'Shows',
 'Sports',
 'Travel & Events',
 'cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16']

In [37]:
# Each category column has the number of appearances of a tag on its corresponding catX column
# The top tags change for each category this is why we need 2 columns per category
categories = pivot.columns.tolist()
cats= pd.DataFrame(columns=cols, index=list(range(1,52)))
i=0
for c in categories:
    cats[c]=pivot[c].sort_values(ascending=False)[:51].values.astype(int).tolist()
    cats['cat'+str(i)]=pivot[c].sort_values(ascending=False)[:51].index.tolist()
    i+=1
cats

,Autos & Vehicles,Comedy,Education,Entertainment,Film & Animation,Gaming,Howto & Style,Movies,Music,News & Politics,...,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16
1,191,4092,875,5031,1103,702,1673,23,3374,2134,...,full movie,Pop,news,Dhadrianwale,funny,animals,science,periyamanaval,football,food
2,161,4060,782,4377,852,455,1305,21,2047,1263,...,ਗੋਲਕ ਬੁਗਨੀ ਬੈਂਕ ਤੇ ਬਟੂਆ,Records,politics,Rara Sahib,comedy,pets,unboxing,priyamanaval episode,nba,street food
3,108,2026,412,2720,811,407,1242,21,1972,1197,...,bugni,music,News,Punjabi,vlog,cats,technology,priyamanaval serial,basketball,travel
4,108,1467,366,2196,778,352,1106,21,1567,1148,...,harish verma,Rap,breaking news,Bhagwant,BuzzFeed,kitten,review,piriyamanaval,highlights,street food videos
5,98,1228,287,2049,603,338,1098,21,1533,847,...,happy raikoti,music video,Politics,Bhai,2018,cat,apple,பிரியமானவள்,sports,cooking
6,84,1216,263,2034,558,321,968,21,1385,668,...,golak bugni bank te batua,rap,CNN,Punjab,fashion,cute,tech,Priyamanaval today,sp:ty=high,street
7,76,1139,262,1906,479,298,953,21,1204,647,...,golak,official,Donald Trump,Bhindran,food,kittens,iphone x,Priyamanaval episode today,soccer,best street food
8,75,1037,262,1839,469,292,861,21,1151,609,...,funny,hip hop,latest news,Bhora,family,wild,MKBHD,Priyamanaval today Episode,Football,the food ranger
9,66,1031,262,1777,450,280,773,21,1114,530,...,official trailer,punjabi songs,donald trump,Channel,hollywood,Cat,iphone,deviamagal,wrestling,food ranger
10,65,995,252,1745,435,241,662,21,1110,519,...,full punjabi movie,Music,latest News,Darbar,comedian,funny cats,nozman,devaimagal,2018,best


In [38]:
# save csv
cats.to_csv("cats.csv", index=False)